In [ ]:
# --- Librerías ---
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow import keras
from datetime import datetime
from google.colab import drive

# Semilla global
tf.random.set_seed(1)

In [ ]:

# --- Montar Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Ruta al CSV ---
dataset = '/content/drive/MyDrive/FINAL_DB.csv'

In [ ]:
# --- Configuración ---
dataset = '/content/drive/MyDrive/FINAL_DB.csv'
label = 'Candidatura Ganadora'
hot = ['Mesa', 'Comunidad', 'Provincia', 'Gobierno Actual de Comunidad', 'Gobierno Actual de España']
norm = ['Edad Media', 'Escolarizacion', 'Esperanza de Vida', 'Habitantes Provincia',
        'Acceso a Internet', 'PIB per capita', 'Criminalidad', 'Inmigracion',
        '% de mujeres', 'Tasa de Pobreza', 'Tasa Desempleo']
epochs = 10
lr = 0.1
momentum = 0
test_ratio = 0.30
metricas = ['accuracy']
inicializacion_pesos = 'glorot_normal'
batch = 32
loss = 'categorical_crossentropy'
optimizador = 'adam'

In [ ]:
# --- Función de carga y preprocesamiento ---
def carga_datos_csv(data, categoria, porcentaje_test=0.25, hot_encoded=None, normalizados=None, seed=None):
    # Normalización
    for col in normalizados:
        if col in data.columns:
            data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())

    # Eliminar filas con NaN
    data = data.dropna()

    # División train/test
    train, test = train_test_split(data, test_size=porcentaje_test, random_state=seed)

    # Separar etiquetas
    label_train = train[categoria].copy()
    label_test = test[categoria].copy()
    train = train.drop(columns=[categoria])
    test = test.drop(columns=[categoria])

    # One-hot encoding de entradas
    if hot_encoded:
        full = pd.concat([train, test])
        full = pd.get_dummies(full, columns=hot_encoded)
        train = full.iloc[:len(train)]
        test = full.iloc[len(train):]

    # Alinear columnas
    test = test.reindex(columns=train.columns, fill_value=0)

    # One-hot encoding de etiquetas
    label_train = pd.get_dummies(label_train)
    label_test = pd.get_dummies(label_test)

    # Alinear columnas de salida
    label_test = label_test.reindex(columns=label_train.columns, fill_value=0)

    return (train.columns, label_train.columns), (
        train.to_numpy().astype(np.float32),
        label_train.to_numpy().astype(np.float32),
        test.to_numpy().astype(np.float32),
        label_test.to_numpy().astype(np.float32)
    )

In [ ]:
# --- Carga de datos ---
data = pd.read_csv(dataset, sep=',')
data = data.dropna(subset=[label])

(x_columns, y_columns), (x_train, y_train, x_test, y_test)= carga_datos_csv(
    data, label, porcentaje_test=test_ratio, hot_encoded=hot, normalizados=norm, seed=int(datetime.utcnow().timestamp())
)


In [ ]:

# --- Red neuronal ---
model = Sequential([
    Dense(len(x_train[0]), input_shape=(len(x_train[0]),)),
    Dense(len(y_train[0]), activation='softmax', kernel_initializer=inicializacion_pesos)
])

model.compile(optimizer=optimizador, loss=loss, metrics=metricas)

In [ ]:
# --- Entrenamiento ---
model.fit(x_train, y_train, epochs=epochs, batch_size=batch)

In [ ]:

# --- Evaluación ---
results = model.evaluate(x_test, y_test, verbose=0)
print('test loss, test acc:', results)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# --- Predicción ---
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# --- Nombres de clases ---
class_names = y_columns

# --- Matriz de confusión ---
cm = confusion_matrix(y_true_classes, y_pred_classes)

print("\nMatriz de confusión:")
print(cm)

# --- Informe de clasificación ---
print("\nInforme de clasificación:")
print(classification_report(y_true_classes, y_pred_classes, target_names=class_names, zero_division=0))

# --- Visualización de la matriz ---
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Matriz de Confusión - Red Neuronal')
plt.tight_layout()
plt.show()
